<a href="https://colab.research.google.com/github/CodeHunterOfficial/ABC_DataMining/blob/main/TimeSeries/TimeSeries-2025/%D0%9B%D0%B0%D0%B1%D0%BE%D1%80%D0%B0%D1%82%D0%BE%D1%80%D0%BD%D1%8B%D0%B9_%D0%BF%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D0%BA%D1%83%D0%BC_%E2%84%96_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# 🧪 **Лабораторный практикум № 3**  
## **Сравнительный анализ классических моделей временных рядов**

**Кафедра:** Кафедра анализа данных и технологий программирования  
**Дисциплина:** Машинное обучение в задачах прогнозирования  
**Уровень:** Магистратура, 2 курс  
**Преподаватель:** Арабов Муллошараф Курбонович  
**Консультации:** 21.09.2025  

---

## 🎯 1. Цели и задачи работы

### **Цель:**  
Освоить полный цикл сравнительного анализа **широкого спектра классических моделей временных рядов** — от линейных (AR, MA) до полу-параметрических (Prophet, TBATS) и многомерных (VAR, VECM). Научиться систематически подбирать преобразования, стратегии прогнозирования, валидировать модели и обоснованно выбирать оптимальную архитектуру для заданного горизонта и структуры данных.

### **Задачи:**
1. Привести временной ряд к стационарному виду с использованием **лог-трансформации**, **преобразования Бокса–Кокса** и **дифференцирования** (обычного, сезонного, комбинированного).
2. Провести **валидацию стационарности** с помощью тестов ADF и KPSS после каждого этапа преобразования.
3. Выполнить **расширенный feature engineering**: лаги, скользящие статистики, временные и циклические признаки, экзогенные переменные.
4. Реализовать и сравнить **три стратегии многопшагового прогнозирования**: рекурсивную, прямую и гибридную.
5. Провести **кросс-валидацию** по схемам: скользящее окно, расширяющееся окно и `TimeSeriesSplit`.
6. Построить и сравнить следующие модели:
   - AR, MA, ARMA,
   - ARIMA, SARIMA, SARIMAX,
   - GARCH,
   - TBATS,
   - VAR, VECM (для многомерных данных),
   - Prophet.
7. Оценить модели по **диагностике адекватности**:
   - Тест Льюнга–Бокса на автокорреляцию остатков,
   - Визуальный и статистический анализ гомоскедастичности,
   - Тест Шапиро–Уилка и Q-Q plot для нормальности ошибок.
8. Сравнить качество прогнозов по метрикам: **MAE, RMSE, MAPE, MASE, SMAPE, R²**, включая сравнение с наивным прогнозом.
9. Разработать **веб-интерфейс** для интерактивного запуска и сравнения моделей.
10. Сформулировать **выводы и практические рекомендации** по выбору модели в зависимости от характеристик ряда, горизонта прогноза и вычислительных ограничений.

> 💡 **Важно:** данная работа **не включает сложные ML-модели** (нейросети, градиентный бустинг). Акцент — на **глубокое понимание классических подходов**, их предпосылок, ограничений и области применимости.

> 📌 **Требования к объёму данных:**  
> - Обучающая выборка: **не менее 500 наблюдений**  
> - Тестовая выборка: **не менее 2 × h** и **минимум 50 точек**  
> Это обеспечивает статистическую надёжность оценок и визуализаций.

> 💡 **Ни одна модель не универсальна. Сравнение по единой методологии — ключ к осознанному выбору.**

---

## 📚 2. Теоретические предпосылки

Классические модели временных рядов основаны на строгих статистических предположениях: стационарность, нормальность остатков, отсутствие автокорреляции. Разные модели охватывают разные структурные компоненты:

- **AR/MA/ARMA** — для стационарных рядов без тренда и сезонности.
- **ARIMA/SARIMA** — для нестационарных рядов с трендом и/или сезонностью.
- **SARIMAX** — расширение SARIMA с экзогенными регрессорами.
- **GARCH** — моделирует **динамику дисперсии**, а не уровня ряда (применима после ARIMA-моделирования остатков).
- **TBATS** — автоматически обрабатывает сложную, кратную и меняющуюся сезонность.
- **VAR/VECM** — применяются к **многомерным** рядам; VECM учитывает долгосрочные равновесия (коинтеграцию).
- **Prophet** — процедурная модель с гибким трендом, встроенной сезонностью и устойчивостью к пропускам.

### 🔑 Ключевые элементы:
- **Преобразование Бокса–Кокса** — параметрическое семейство, включающее лог-трансформацию (λ → 0), стабилизирующее дисперсию (`scipy.stats.boxcox`, `boxcox_normmax`).
- **Кросс-валидация во временных рядах** — запрещено перемешивание; используется `sklearn.model_selection.TimeSeriesSplit`.
- **Диагностика моделей** — неотъемлемая часть: только «белый шум» в остатках гарантирует адекватность.
- **Стратегии прогнозирования** — рекурсивная склонна к накоплению ошибки, прямая требует больше данных.

---

## 🧪 3. Методика эксперимента и порядок выполнения

### **3.1. Этап 1. Предобработка и приведение к стационарности**

**Задача:** Стабилизировать дисперсию и устранить тренд/сезонность.

#### 📌 Что нужно сделать:
- Применить **лог-трансформацию** (если данные > 0).
- Выполнить **преобразование Бокса–Кокса**:
  - Подобрать λ через `scipy.stats.boxcox_normmax`,
  - Сохранить λ для обратного преобразования.
- Выполнить **дифференцирование**:
  - 1-го порядка,
  - Сезонное (`s = 7, 12, 30`),
  - Комбинированное (например, `diff(1).diff(7)`).
- Проверить стационарность после каждого шага:
  - **ADF-тест** (`statsmodels.tsa.stattools.adfuller`),
  - **KPSS-тест** (`statsmodels.tsa.stattools.kpss`).
- Выбрать **оптимальную цепочку преобразований**.

> 📌 Обратное преобразование обязательно для оценки метрик в исходных единицах.

---

### **3.2. Этап 2. Расширенный feature engineering**

**Задача:** Создать информативные признаки для моделей с регрессорами.

#### 📌 Что нужно сделать:
- **Временные признаки**: день недели, месяц, квартал, индикатор выходных.
- **Циклическое кодирование**: `sin(2π·t/T)`, `cos(2π·t/T)`.
- **Лаговые признаки**: `lag_1`, `lag_7`, `lag_30`.
- **Скользящие статистики**: среднее, std, min, max по окнам 7, 30, 90.
- **Экзогенные переменные** (если доступны): праздники, погода, промо-акции.

> 🛠️ Инструменты: `pandas`, `numpy`.

---

### **3.3. Этап 3. Стратегии многопшагового прогнозирования**

**Задача:** Реализовать три подхода к прогнозу на `h ≥ 7` шагов.

#### 📌 Что нужно сделать:
- **Рекурсивная стратегия**: итеративное использование прогнозов.
- **Прямая стратегия**: отдельная модель для каждого шага.
- **Гибридная стратегия**: например, рекурсивная для `h ≤ 3`, прямая — для `h > 3`.
- Сравнить по:
  - MAE/RMSE на каждом шаге,
  - Времени вычислений,
  - Стабильности ошибки во времени.

> 💡 Для SARIMAX, Prophet, TBATS используется нативный многопшаговый прогноз.

---

### **3.4. Этап 4. Кросс-валидация для временных рядов**

**Задача:** Оценить качество без утечки будущего.

#### 📌 Что нужно сделать:
- Реализовать **скользящее окно** (фиксированная длина обучения).
- Реализовать **расширяющееся окно**.
- Настроить **TimeSeriesSplit** (`sklearn.model_selection`).
- Для каждой схемы:
  - Оценить среднее качество по фолдам,
  - Проанализировать динамику ошибки.

> 📊 Визуализируйте ошибку по фолдам для всех моделей.

---

### **3.5. Этап 5. Построение и настройка моделей**

**Задача:** Построить 11 моделей с оптимальными параметрами.

#### 📌 Что нужно сделать:
- **AR/MA/ARMA**: подбор порядков по ACF/PACF или AIC.
- **ARIMA/SARIMA**: через `pmdarima.auto_arima` с ограничениями.
- **SARIMAX**: включить экзогенные признаки.
- **GARCH**: применить к остаткам ARIMA (через `arch` или `statsmodels`).
- **TBATS**: использовать `tbats.TBATS` с автоматическим определением сезонности.
- **VAR/VECM**: для многомерных данных; проверить стационарность и коинтеграцию.
- **Prophet**: настроить сезонность, праздники, точки смены тренда.
- Для всех моделей:
  - Построить прогноз на `h` шагов,
  - Рассчитать доверительные интервалы (если поддерживается),
  - Оценить качество на тестовой выборке,
  - Сравнить с **наивным прогнозом**.

> 🛠️ Инструменты: `statsmodels`, `pmdarima`, `prophet`, `tbats`, `arch`.

---

### **3.6. Этап 6. Диагностика адекватности моделей**

**Задача:** Проверить, являются ли остатки «белым шумом».

#### 📌 Что нужно сделать:
- **Тест Льюнга–Бокса** (`statsmodels.stats.diagnostic.acorr_ljungbox`) — p > 0.05.
- **Гомоскедастичность**: график остатков vs прогнозов; тест Бреуша–Пагана (опционально).
- **Нормальность ошибок**: Q-Q plot + тест Шапиро–Уилка (`scipy.stats.shapiro`).
- Визуализировать:
  - Остатки во времени,
  - ACF/PACF остатков,
  - Гистограмму и плотность распределения.

---

### **3.7. Этап 7. Веб-интерфейс для интерактивного сравнения моделей**

**Задача:** Создать инструмент для визуального анализа.

#### 🎛️ Обязательные элементы:
- Загрузка CSV/Parquet-файла
- Выбор целевой переменной и **горизонта h** (7, 30, 90)
- Выбор преобразования: лог, Бокс–Кокс (с ручным/автоматическим λ)
- Переключение между моделями (ARIMA, Prophet, TBATS и др.)
- **Визуализация прогнозов с доверительными интервалами**
- **Таблица сравнения метрик**: MAE, RMSE, MAPE, MASE, SMAPE, R², время обучения
- Экспорт прогноза и параметров модели

#### 🧰 Технологический стек:
- `Streamlit` + `Plotly` (интерактив),
- `statsmodels`, `pmdarima`, `prophet`, `scipy`, `sklearn`.

> 🌐 Деплой на Hugging Face Spaces или Render — приветствуется.

---

### **3.8. Этап 8. Сравнительный анализ и выводы**

**Задача:** Обосновать выбор лучшей модели.

#### 📌 Что нужно сделать:
- Собрать **сводную таблицу** по всем моделям: параметры, метрики, p-value тестов, время выполнения.
- Проанализировать:
  - Какая модель лучше на коротком/длинном горизонте?
  - Насколько преобразование Бокса–Кокса влияет на качество?
  - Согласуется ли кросс-валидация с финальной оценкой?
  - Какие модели дают адекватные остатки?
- Сформулировать **практические рекомендации**.
- Написать **рефлексию**: сложности, неожиданные инсайты, ограничения.

---

## 🔍 4. Дополнительные исследовательские задания (по желанию)

1. **Анализ чувствительности к λ** в преобразовании Бокса–Кокса.
2. **Прогнозные интервалы** и их **эмпирическое покрытие** на тесте.
3. **Ансамбли классических моделей** (усреднение прогнозов ARIMA, Prophet, TBATS).
4. **Автоматическое обнаружение структурных разрывов** и адаптация моделей.
5. **Байесовская оптимизация** гиперпараметров SARIMAX.

---

## 📄 5. Требования к отчёту

Отчёт должен содержать:

1. Титульный лист (ФИО, группа, дата, подпись преподавателя)  
2. Описание источника данных и структуры датасета  
3. Результаты предобработки и выбор цепочки преобразований  
4. Примеры feature engineering с кодом  
5. Сравнение трёх стратегий прогнозирования  
6. Результаты кросс-валидации по двум схемам  
7. Подбор параметров и построение всех 11 моделей  
8. Сравнительная таблица метрик и времени выполнения  
9. Результаты **диагностики адекватности**:  
   - p-value теста Льюнга–Бокса,  
   - Графики остатков на гомоскедастичность,  
   - Выводы по нормальности ошибок.  
10. Скриншоты или ссылка на веб-интерфейс  
11. Выводы и рефлексия

---

## 📊 6. Критерии оценки

| Оценка | Критерии |
|--------|---------|
| **5** | Полное выполнение всех этапов. Корректные преобразования и диагностика. Все 11 моделей реализованы и сравнены. Глубокий анализ стратегий и кросс-валидации. Работающий веб-интерфейс с полной функциональностью. Выполнены доп. задания. |
| **4** | Реализованы основные модели (ARIMA, SARIMAX, Prophet, TBATS). Есть сравнение метрик и диагностика. Веб-интерфейс базовый. Отчёт полный. |
| **3** | Построены 4–6 моделей. Есть оценка качества, но без комплексной диагностики и стратегий. |
| **2** | Работа не выполнена или отсутствует содержательный анализ. |

---

## 📖 7. Литература

1. Hyndman, R.J., Athanasopoulos, G. *Forecasting: Principles and Practice*. — 3rd ed., 2021. [https://otexts.com/fpp3/](https://otexts.com/fpp3/)  
2. Box, G.E.P., Jenkins, G.M. *Time Series Analysis: Forecasting and Control*. — 5th ed., 2015.  
3. Документация:
   - `scipy.stats.boxcox`, `boxcox_normmax` — [SciPy Docs](https://docs.scipy.org/doc/scipy/reference/stats.html)  
   - `statsmodels.tsa.arima.model.ARIMA`, `ExponentialSmoothing` — [Statsmodels Docs](https://www.statsmodels.org/stable/index.html)  
   - `pmdarima.auto_arima` — [pmdarima](https://alkaline-ml.com/pmdarima/)  
   - `prophet`, `tbats` — [Prophet](https://facebook.github.io/prophet/), [TBATS](https://pypi.org/project/tbats/)  
   - `sklearn.model_selection.TimeSeriesSplit` — [Scikit-learn Docs](https://scikit-learn.org)  
   - `streamlit`, `plotly` — [Streamlit](https://docs.streamlit.io/), [Plotly](https://plotly.com/python/)  

> 💡 Эта работа завершает цикл по классическим методам и создаёт основу для перехода к **современным ML-подходам** (нейросети, трансформеры) в последующих лабораторных работах.

